Drawn from https://www.statsmodels.org/dev/examples/notebooks/generated/glm_formula.html.

In [2]:
# Summon libraries.
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

Call data.

In [3]:
# Call data. Uses PANDAS.
novel = pd.read_csv('../../data/Exercise3.3Data.csv')
# Turns success into 0/1.
novel['success'] = novel['success'].map({'yes' : 1, 'no' : 0})

# Since each model has identical formula, develop it once.
formula = 'success ~ cover + C(methods,Treatment("none")) + C(novels,Treatment("many")) + years'

Logit models.

In [3]:
fitted_logit = smf.logit(formula = formula, data=novel).fit()
print(fitted_logit.summary())
fitloglike_logit = (fitted_logit.llf)

Optimization terminated successfully.
         Current function value: 0.364448
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                success   No. Observations:                   44
Model:                          Logit   Df Residuals:                       37
Method:                           MLE   Df Model:                            6
Date:                Thu, 26 Oct 2023   Pseudo R-squ.:                  0.4670
Time:                        11:08:16   Log-Likelihood:                -16.036
converged:                       True   LL-Null:                       -30.088
Covariance Type:            nonrobust   LLR p-value:                 8.979e-05
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                  -6.8762

In [4]:
# The three models don't vary in how many parameters they produce, 
# Thus, we can set model_param to fitted.df_model once.
# df_model is number of regressors. Add 1 to get number of parameters.
# Keep df_model for degrees of freedom.
model_param = fitted_logit.df_model + 1
deg_free = fitted_logit.df_model

Logit AIC, BIC, AICC.

In [5]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_logit = fitted_logit.aic
aicc_logit = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitloglike_logit,
# Number of observations.
    fitted_logit.nobs,
    model_param)
bic_logit = fitted_logit.bic

In [6]:
# Null model.
null = smf.logit('success ~ 1', data=novel).fit()
nullloglike_logit = (null.llf)

Optimization terminated successfully.
         Current function value: 0.683821
         Iterations 4


Logit log likelihood. Found the names through dir() https://stackoverflow.com/questions/2675028/list-attributes-of-an-object.

In [7]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (nullloglike_logit-(fitloglike_logit))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 28.104794170180533.
p-value is 8.978739912679501e-05.


Logit prediction.

In [8]:
# Prediction.
predict_val = pd.DataFrame(
    {"cover" : "yes", "methods" : "none", "novels" : "first" , "years" : 0}, index=[0])
predict_val = sm.add_constant(predict_val)
# Simpler.
print(f'Predicted: {fitted_logit.predict(predict_val)}')

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_logit.predict(predict_val).values[0]}')

Predicted: 0    0.186086
dtype: float64
Predicted: 0.18608601384386772


Probit models.

In [9]:
formula = 'success ~ cover + C(methods,Treatment("none")) + C(novels,Treatment("many")) + years'
fitted_probit = smf.probit(formula=formula, data=novel).fit()
print(fitted_probit.summary())
fitloglike_probit = (fitted_probit.llf)

Optimization terminated successfully.
         Current function value: 0.369389
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                success   No. Observations:                   44
Model:                         Probit   Df Residuals:                       37
Method:                           MLE   Df Model:                            6
Date:                Thu, 26 Oct 2023   Pseudo R-squ.:                  0.4598
Time:                        11:08:16   Log-Likelihood:                -16.253
converged:                       True   LL-Null:                       -30.088
Covariance Type:            nonrobust   LLR p-value:                 0.0001084
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                  -3.6827

In [10]:
# Null model.
null_probit = smf.probit('success ~ 1', data=novel).fit()
nullloglike_probit = (null_probit.llf)

Optimization terminated successfully.
         Current function value: 0.683821
         Iterations 4


Probit log likelihood.

In [11]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (nullloglike_probit-(fitloglike_probit))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 27.670024452307146.
p-value is 0.0001084039868023412.


Probit AIC, BIC, AICC.

In [12]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_probit = fitted_probit.aic
aicc_probit = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitloglike_probit,
# Number of observations.
    fitted_probit.nobs,
    model_param)
bic_probit = fitted_probit.bic

Probit prediction.

In [13]:
# Prediction.
predict_val = pd.DataFrame(
    {"cover" : "yes", "methods" : "none", "novels" : "first" , "years" : 0}, index=[0])
predict_val = sm.add_constant(predict_val)
# Simpler.
print(f'Predicted: {fitted_probit.predict(predict_val)}')

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_probit.predict(predict_val).values[0]}')

Predicted: 0    0.199526
dtype: float64
Predicted: 0.19952619997183002


Cloglog models. Found format through https://github.com/statsmodels/statsmodels/issues/827. 

In [14]:
# Cloglog. 
# Note that this uses a binomial family with cloglog link, akin to R.
# Furthermore, this similar format can be used for Probit and Logit, just changing the link function.
# It just happens there's no smf.cloglog.
formula = 'success ~ cover + C(methods,Treatment("none")) + C(novels,Treatment("many")) + years'
fitted_cloglog = smf.glm(formula=formula, data=novel,family=sm.families.Binomial(sm.families.links.CLogLog ())).fit()
print(fitted_cloglog.summary())
fitloglike_cloglog = (fitted_cloglog.llf)

# Null model.
null_cloglog = smf.glm(formula = 'success ~ 1', data=novel, family=sm.families.Binomial(sm.families.links.CLogLog ())).fit()
nullloglike_cloglog = (null_cloglog.llf)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                   44
Model:                            GLM   Df Residuals:                       37
Model Family:                Binomial   Df Model:                            6
Link Function:                CLogLog   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -15.399
Date:                Thu, 26 Oct 2023   Deviance:                       30.797
Time:                        11:08:16   Pearson chi2:                     47.6
No. Iterations:                    13   Pseudo R-squ. (CS):             0.4871
Covariance Type:            nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

Cloglog log likelihood.

In [15]:
# Uses null and fitted log likelihoods to perform the deviance test.
deviance= -2 * (nullloglike_cloglog-(fitloglike_cloglog))
print(f"Deviance statistic is {deviance}.")
# Chi2.cdf is from scipy.stats.
from scipy.stats import chi2
pvalue = 1 - chi2.cdf(deviance,deg_free)
print(f"p-value is {pvalue}.")

Deviance statistic is 29.379122948607332.
p-value is 5.156521244931156e-05.


Cloglog AIC, BIC, AICC.

In [16]:
# AIC, BIC are built into the regression. 
# AICC needs to be developed.
aic_cloglog = fitted_cloglog.aic
aicc_cloglog = sm.tools.eval_measures.aicc(
# Value of log likelihood function.
    fitloglike_cloglog,
# Number of observations.
    fitted_cloglog.nobs,
    model_param)
# statsmodel alludes to using a deviance-based bic.
# _llf makes it use a log-likelihood function-based bic.
bic_cloglog = fitted_cloglog.bic_llf

Cloglog prediction. https://www.statology.org/statsmodels-predict/

In [17]:
prediction = pd.DataFrame({"cover" : "yes", "methods" : "none", "novels" : "first" , "years" : 0},index=[0])

In [18]:
# Prediction.
predict_val = sm.add_constant(prediction)
# Simpler.
print(f'Predicted: {fitted_cloglog.predict(predict_val)}')

# Isolated. This one grabs the item "values" from the array.
print(f'Predicted: {fitted_cloglog.predict(predict_val).values[0]}')

Predicted: 0    0.200779
dtype: float64
Predicted: 0.20077857938007804


Model comparisons. From https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/

In [19]:
data = [[aic_logit,aicc_logit,bic_logit],
        [aic_probit,aicc_probit,bic_probit],
        [aic_cloglog,aicc_cloglog,bic_cloglog]]

comparison = pd.DataFrame(data,
                          columns = ['AIC','AICC','BIC'],
                          index = ['logit','probit','cloglog'])

comparison

,AIC,AICC,BIC
logit,46.071421,49.182532,58.560749
probit,46.506191,49.617302,58.995518
cloglog,44.797092,47.908203,57.286420


Automated comparison. 
https://www.geeksforgeeks.org/get-minimum-values-in-rows-or-columns-with-their-index-position-in-pandas-dataframe/

In [20]:
# Initialize.
logit = 0
probit = 0
cloglog = 0 

# Runs through all 3 columns.
# If column has lowest value, that model +1 point.
# Model with highest points is declared at end.
def points(cur_min,logit,probit,cloglog):
        if cur_min == "logit":
                logit += 1
        elif cur_min == "probit":
                probit += 1
        else:
                cloglog += 1 
        return(logit,probit,cloglog)

# Runs through the models.
cur_min = comparison['AIC'].idxmin()
logit,probit,cloglog = points(cur_min,logit,probit,cloglog)
cur_min = comparison['AICC'].idxmin()
logit,probit,cloglog = points(cur_min,logit,probit,cloglog)
cur_min = comparison['BIC'].idxmin()
logit,probit,cloglog = points(cur_min,logit,probit,cloglog)

final = {logit:"logit",probit:"probit",cloglog:"cloglog"}

print(f"The model with the best fit is the {final.get(max(final))} model.")

The model with the best fit is the cloglog model.
